In [ ]:
# 1. 필요한 패키지 가져오는 코드 
import tensorflow as tf   # 텐써 플로우 2.0 
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential  # 모델을 구성하기 위한 모듈
from tensorflow.keras.layers import Activation,Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout  # CNN, 완전 연결계층을 구성하기 위한 모듈
from tensorflow.keras.utils import to_categorical # one encoding 하는 모듈
import numpy as np

# 2. vgg 신경망 설계도를 가져옵니다. 
from  tensorflow.keras.applications  import  VGG16
vgg16 = VGG16( weights = 'imagenet', input_shape =(32,32,3), include_top=False )

In [ ]:
#■ vgg16 신경망을 동결 시켰을때 
#for  layer  in  vgg16.layers[:]:
#    layer.trainable = False   # False 는 동결

#vgg16.summary()

#■ vgg16 신경망을 동결 시키지 않았을때 
for  layer  in  vgg16.layers[:]:
    layer.trainable = True   # True 는 동결 해제 (default)

vgg16.summary()

In [ ]:
# 3. cifar10 데이터를 가져옵니다.

tf.random.set_seed(777)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape) # (50000, 32, 32)
print(x_test.shape)  # (10000, 32, 32)   

# 설명:  위의 결과는 색조를 나타내는 차원이 없습니다. 
# 그래서 다음과 같이 색조를 나타내는 차원을 추가주겠금 reshape 를 해줘야합니다.
# (50000, 32, 32)  ---->  (50000, 32, 32, 1)
# (10000, 32, 32)  ---->  (10000, 32, 32, 1)
# 3차원을 4차원으로 변경해주되 안의 요소(픽셀)의 갯수는 동일해야합니다. 
# 3차원(50000*32*32) ------> 4차원( 50000*32*32*1)  로 갯수는 동일합니다 

x_train = x_train.reshape(-1, 32, 32, 3)  # -1의 의미는 학습데이터는 갯수만큼 알아서 하고 4차원으로 변경해라
x_test = x_test.reshape(-1, 32, 32, 3)
print(x_train.shape) # (50000, 32, 32, 3)
print(x_test.shape)  # (10000, 32, 32, 3)

In [ ]:
# 4. 정규화 진행  
x_train = x_train / 255 
x_test = x_test / 255

# 5. 정답 데이터를 준비한다. 
# 하나의 숫자를 one hot encoding 한다. (예:  4 ---> 0 0 0 0 1 0 0 0 0 0 ) 
y_train = to_categorical(y_train)  # 훈련 데이터의 라벨(정답)을 원핫 인코딩
y_test = to_categorical(y_test)    # 테스트 데이터의 라벨(정답)을 원핫 인코딩 

In [ ]:
# 6. 모델을 구성합니다. 

model = Sequential()
model.add(vgg16)  # vgg16 모델을 사용합니다.

#완전 연결계층은 직접 구현합니다. 
model.add( Flatten() )
model.add( Dense(256) )
model.add(BatchNormalization() )
model.add(Dropout(0.4) )
model.add(Activation('relu') )
model.add( Dense(256) )
model.add(BatchNormalization() )
model.add(Dropout(0.4) )
model.add(Activation('relu') )
model.add(Dense(10, activation='softmax') ) 
model.summary()

# 이미지넷 대회에서의 vgg 신경망의 출력층의 뉴런의 갯수는 1000개 입니다.
# 그런데 우리는 cifar10 데이터로 10개의 클래스를 분류할거라서 include_top=False 로 두고
# 완전 연결계층을 우리 손으로 직접 만들었습니다. 

# 7. 모델을 설정합니다. ( 경사하강법, 오차함수를 정의해줍니다. )
model.compile(optimizer='adam', 
                     loss = 'categorical_crossentropy', 
                     metrics=['acc'])  # 학습과정에서 정확도를 보려고 

In [ ]:
from  tensorflow.keras.callbacks  import  EarlyStopping 

'''
# 얼리 스탑 기능 추가
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["val_acc"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

#검증 데이터의 정확도가 80% 넘으면 조기중단하게 설정
callbacks  = MyThresholdCallback(threshold=0.8)
'''

#8. 모델을 훈련시킵니다. 

history = model.fit(x_train, y_train, 
                         epochs = 100,  # 30에폭
                         batch_size = 100,
                         validation_data=(x_test, y_test) )
                        #, callbacks = callbacks )

In [ ]:
# 9.모델을 평가합니다. (오차, 정확도가 출력됩니다.)
model.evaluate(x_test, y_test)

train_acc_list=history.history['acc']
train_acc_list

test_acc_list=history.history['val_acc']
test_acc_list

In [ ]:
import  matplotlib.pyplot  as  plt

x = np.arange( len(train_acc_list) )
plt.plot( x, train_acc_list, label='train acc')
plt.plot( x, test_acc_list, label='test acc',  linestyle='--')
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()